In [1]:
!pip3 install -U -q datasets transformers accelerate bitsandbytes hnswlib peft wandb openai-whisper ffmpeg-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 22.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 112.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.3/20.3 MB 92.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.9 MB/s eta 0:00:00
   ━━━━━

In [3]:
import torch

from sentence_transformers import SentenceTransformer, CrossEncoder, util
from transformers import (AutoModelForCausalLM, AutoTokenizer, pipeline,
                          BitsAndBytesConfig, TrainingArguments, Trainer,
                          DataCollatorForLanguageModeling)
from datasets import load_dataset
import datasets
from pathlib import Path
datasets.config.DOWNLOADED_DATASETS_PATH = Path('/home/hpc/b110dc/b110dc16/datasets')

from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model

from huggingface_hub import login
import wandb

import hnswlib

import os
import matplotlib.pyplot as plt
import re
import numpy as np
from tqdm import tqdm

from gensim.models.word2vec import Word2Vec

from IPython.display import HTML, Audio
# from google.colab.output import eval_js
from base64 import b64decode
from scipy.io.wavfile import read as wav_read
import scipy
import io
import ffmpeg

import whisper

In [6]:
semb_model = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')

In [7]:
name = "./finedtuned_model"

tokenizer = AutoTokenizer.from_pretrained(name)
tokenizer.pad_token = tokenizer.eos_token

adapter_configs = {
    'target_modules': 'all-linear',
    'lora_alpha': 2,
    'lora_dropout': 0.05,
    'r': 4,
    'bias': 'none',
    'task_type': 'CAUSAL_LM'
}

quantization_config = BitsAndBytesConfig(load_in_4bit=True,
                                         bnb_4bit_compute_dtype=torch.float16,
                                         bnb_4bit_quant_type="nf4",
                                         bnb_4bit_use_double_quant=True)

loaded_model = AutoModelForCausalLM.from_pretrained(name, device_map="auto",
                                             bos_token_id=tokenizer.bos_token_id,
                                             eos_token_id=tokenizer.eos_token_id,
                                             quantization_config=quantization_config)

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [5]:
query = ""

Archive:  ft_model.zip
   creating: finedtuned_model/
  inflating: finedtuned_model/training_args.bin  
  inflating: finedtuned_model/vocab.json  
   creating: finedtuned_model/.ipynb_checkpoints/
  inflating: finedtuned_model/adapter_model.safetensors  
  inflating: finedtuned_model/tokenizer_config.json  
  inflating: finedtuned_model/adapter_config.json  
  inflating: finedtuned_model/special_tokens_map.json  
  inflating: finedtuned_model/README.md  
  inflating: finedtuned_model/tokenizer.json  
  inflating: finedtuned_model/merges.txt  


In [8]:
ds = load_dataset("medalpaca/medical_meadow_medical_flashcards")
arr_len = np.array([ len(arr) for arr in ds['train']['input'] ], dtype=np.int16)
filter_arr = np.argwhere(arr_len > 2).reshape(-1,)
ds = ds['train'].select(filter_arr)


index_path = './hnswlib_filter.index'
corpus = np.array(ds['output'])
print("Creating a HNSWLIB index")
semb_model = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')
corpus_embeddings = semb_model.encode(corpus, convert_to_tensor=True,
                                    show_progress_bar=True)
index = hnswlib.Index(space='cosine', dim=corpus_embeddings.size(1))
index.init_index(max_elements=corpus_embeddings.size(0),
               ef_construction=256, M=64)
index.add_items(corpus_embeddings.cpu(), list(range(len(corpus_embeddings))))



README.md:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

(…)l_meadow_wikidoc_medical_flashcards.json:   0%|          | 0.00/17.7M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/33955 [00:00<?, ? examples/s]

Creating a HNSWLIB index


Batches:   0%|          | 0/1049 [00:00<?, ?it/s]

In [51]:

def retrive_context(query, semb_model=semb_model, index=index):
    query_embedding = semb_model.encode(query, convert_to_tensor=True)
    ids, distances = index.knn_query(query_embedding.cpu(), k=32)
    # context = ds['output'][ids[0][np.argsort(-distances[0]) ][:5] ]
    context_list = [ds['output'][i] for i in ids[0][:5]]
    context = ' '.join(context_list)
    return context

In [60]:
prompt_js = """
Instruction: You are an interactive question answering chatbot and answer elaborately questions based on the instruction and context provided. \n
{} based on the context "{}". Do not repeat the answers and do not keep the answer the same as the context. Summarize or elaborate on the context where needed.

Question: {}
"""

sample = 10
query = ds['input'][sample]
inst = ds['instruction'][sample]
context = retrive_context(query)
prompt = prompt_js.format(inst, context, query)

prompt = "What is a T-test?"
inputs = tokenizer(prompt, return_tensors='pt').to("cuda")

output = loaded_model.generate(inputs['input_ids'],
    max_new_tokens=500,  # Set max tokens for the output
    min_length=20,   # Ensure at least 50 tokens are generated
    eos_token_id=tokenizer.eos_token_id,
    num_beams=2,     # Use beam search for better quality
    top_p=0.95,      # Use nucleus sampling
    temperature=0.5  # Adjust randomness
)

output_text = tokenizer.decode(output[0])
print(output_text)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


What is a T-test?

A T-test is a test that is used to determine if a given test is correct. A T-test is a test that is used to determine if a given test is correct. A T-test is a test that is used to determine if a given test is correct. A T-test is a test that is used to determine if a given test is correct. A T-test is a test that is used to determine if a given test is correct. A T-test is a test that is used to determine if a given test is correct. A T-test is a test that is used to determine if a given test is correct. A T-test is a test that is used to determine if a given test is correct. A T-test is a test that is used to determine if a given test is correct. A T-test is a test that is used to determine if a given test is correct. A T-test is a test that is used to determine if a given test is correct. A T-test is a test that is used to determine if a given test is correct. A T-test is a test that is used to determine if a given test is correct. A T-test is a test that is used 